## Designing Subnets

In this notebook you will: 

-   Reserve resources for this experiment
-   Configure your reserved resources
-   Access your reserved resources over SSH
-   Delete your FABRIC reservation (in case you finish early)

### Exercise: Reserve resources

In this exercise, we will reserve resources on FABRIC: six hosts connected by a three routers.

In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager
fablib = fablib_manager() 
conf = fablib.show_config()

In [ ]:
!chmod 600 {fablib.get_bastion_key_filename()}
!chmod 600 {fablib.get_default_slice_private_key_file()}

In [ ]:
import os
slice_name="designing-subnets_" + os.getenv('NB_USER')

In [ ]:
try:
    slice = fablib.get_slice(slice_name)
    print("You already have a slice by this name!")
    print("If you previously reserved resources, skip to the 'log in to resources' section.")
except:
    print("You don't have a slice named %s yet." % slice_name)
    print("Continue to the next step to make one.")
    slice = fablib.new_slice(name=slice_name)

Next, we’ll select a random FABRIC site for our experiment. We’ll make sure to get one that has sufficient capacity for the experiment we’re going to run.

Once we find a suitable site, we’ll print details about available resources at this site.

In [ ]:
exp_requires = {'core': 9*2, 'nic': 12}
while True:
    site_name = fablib.get_random_site()
    if ( (fablib.resources.get_core_available(site_name) > 1.2*exp_requires['core']) and
        (fablib.resources.get_component_available(site_name, 'SharedNIC-ConnectX-6') > 1.2**exp_requires['nic']) ):
        break

fablib.show_site(site_name)

In [ ]:
# this cell sets up the hosts and routers
node_names = ["romeo", "router-a","juliet","othello","router-b","desdemona","hamlet","router-c","ophelia"]
for n in node_names:
    slice.add_node(name=n, site=site_name, cores=2, ram=4, disk=10, image='default_ubuntu_20')

In [ ]:
# this cell sets up the network links
nets = [
    {"name": "net0",  "nodes": ["router-a","router-b","router-c"]},
    {"name": "net1",  "nodes": ["romeo", "router-a","juliet"]},
    {"name": "net2",  "nodes": ["othello", "router-b","desdemona"]},
    {"name": "net3",  "nodes": ["ophelia", "router-c","hamlet"]} 
] 

for n in nets:
    ifaces = [slice.get_node(node).add_component(model="NIC_Basic", name=n["name"]).get_interfaces()[0] for node in n['nodes'] ]
    slice.add_l2network(name=n["name"], type='L2Bridge', interfaces=ifaces)

The following cell submits our request to the FABRIC site. The output of this cell will update automatically as the status of our request changes.

While it is being prepared, the “State” of the slice will appear as “Configuring”.
When it is ready, the “State” of the slice will change to “StableOK”.

In [ ]:
slice.submit()

In [ ]:
slice.wait_ssh(progress=True)

### Exercise: Configure resources

Next, we need to configure our resources - assign IP addresses to network interfaces, enable forwarding on the routers, and install any necessary software.

First, we’ll configure IP addresses and add the IP addresses and hostnames to the host files:

In [ ]:
from ipaddress import ip_address, IPv4Address, IPv4Network

if_conf = {
    "router-a-net0-p1":  {"addr": "10.10.100.1", "subnet": "10.10.100.0/24", "hostname": "router-a"},
    "router-b-net0-p1":  {"addr": "10.10.100.2", "subnet": "10.10.100.0/24", "hostname": "router-b"},
    "router-c-net0-p1":  {"addr": "10.10.100.3", "subnet": "10.10.100.0/24", "hostname": "router-c"}, 
    "romeo-net1-p1":  {"addr": "0.0.0.0", "subnet": "0.0.0.0/24", "hostname": "romeo"},
    "router-a-net1-p1":  {"addr": "0.0.0.0", "subnet": "0.0.0.0/24", "hostname": "router-a"},
    "juliet-net1-p1":  {"addr": "0.0.0.0", "subnet": "0.0.0.0/24", "hostname": "juliet"},
    "othello-net2-p1":  {"addr": "0.0.0.0", "subnet": "0.0.0.0/24", "hostname": "othello"},
    "router-b-net2-p1":  {"addr": "0.0.0.0", "subnet": "0.0.0.0/24", "hostname": "router-b"},
    "desdemona-net2-p1":  {"addr": "0.0.0.0", "subnet": "0.0.0.0/24", "hostname": "desdemona"},
    "hamlet-net3-p1":  {"addr": "0.0.0.0", "subnet": "0.0.0.0/24", "hostname": "hamlet"},
    "router-c-net3-p1":  {"addr": "0.0.0.0", "subnet": "0.0.0.0/24", "hostname": "router-c"},
    "ophelia-net3-p1":  {"addr": "0.0.0.0", "subnet": "0.0.0.0/24", "hostname": "ophelia"}
}

for iface in slice.get_interfaces():
    if_name = iface.get_name()
    hostname = if_conf[if_name]['hostname']
    iface.ip_addr_add(addr=if_conf[if_name]['addr'], subnet=IPv4Network(if_conf[if_name]['subnet']))


In [ ]:
# for link, config in if_conf.items():
#     addr = config['addr']
#     hostname = config['hostname']
#     for node in slice.get_nodes():
#         node.execute(f"echo '{addr}\t{hostname}' | sudo tee -a /etc/hosts > /dev/null")

Let’s make sure that all of the network interfaces are brought up:

In [ ]:
for iface in slice.get_interfaces():
    iface.ip_link_up()

And, we’ll enable IP forwarding on the router:

In [ ]:
for n in ['router-a','router-b','router-c']:
    slice.get_node(name=n).execute("sudo sysctl -w net.ipv4.ip_forward=1")

### Exercise: Log in to resources
Now, we are finally ready to log in to our resources over SSH! Run the following cells, and observe the table output - you will see an SSH command for each of the nodes in your topology.

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
ssh_str = 'ssh -i ' + slice.get_slice_private_key_file() + \
    ' -J ' + fablib.get_bastion_username() + '@' + fablib.get_bastion_public_addr() + \
    ' -F /home/fabric/work/fabric_config/ssh_config '
slice_info = [{'Name': n.get_name(), 'SSH command': ssh_str + n.get_username() + '@' + str(n.get_management_ip())} for n in slice.get_nodes()]
pd.DataFrame(slice_info).set_index('Name')

Now, you can open an SSH session on any of the nodes as follows:

-   In Jupyter, from the menu bar, use File \> New \> Terminal to open a new terminal.
-   Copy an SSH command from the table, and paste it into the terminal. (Note that each SSH command is a single line, even if the display wraps the text to a second line! When you copy and paste it, paste it all together.)

You can repeat this process (open several terminals) to start a session on each host and the router. Each terminal session will have a tab in the Jupyter environment, so that you can easily switch between them.


When you have logged in to each node, continue to design subnets challenge.

### Delete resources

If you finished your experiment early, you should delete your slice! The following cell deletes all the resources in your slice, freeing them for other experimenters.

In [ ]:
slice.delete()

In [ ]:
slice.show()